In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,log_loss
from mlxtend.plotting import plot_decision_regions
import sys, math, os, json, re, random
import scipy.stats as stats
import seaborn as sns
import datetime as dt

from reticulum import AdaptiveBayesianReticulum

# jupyter magic to display plots directly in the notebook
%matplotlib inline

# use vector graphics format for nicer plots
%config Inline.Backend.figure_format = 'svg'

%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2

In [61]:
training = pd.read_excel('data/RfqData.xlsx',sheet_name = 'Training RFQs')
test = pd.read_excel('data/RfqData.xlsx',sheet_name = 'OOS RFQs')
competition = pd.read_excel('data/RfqData.xlsx',sheet_name = 'Competition RFQs')

In [3]:
target_mapping = {'MISSED': 0, 'DONE': 1}
side_mapping = {'Offer': -1, 'Bid': 1}

## Simple First

In [42]:
def simple_features(df):
    df['delta_from_mid'] = abs(df['MidPrice'] - df['QuotedPrice'])
    df['Traded'] = df['Traded'].map(target_mapping)
    df['Side'] = df['Side'].map(side_mapping)
    df['PnL']=((df['NextMidPrice']-df['QuotedPrice']>0).astype(float)*2-1)*df['Side']
    df = df[['delta_from_mid','Notional','Traded','PnL']]
    return df

In [62]:
simple_train = simple_features(training)
simple_test = simple_features(test)
simple_train.head()

,delta_from_mid,Notional,Traded,PnL
0,0.24,10000000,0,1.0
1,0.01,1000,1,1.0
2,0.10,1000,0,-1.0
3,0.02,20000,1,-1.0
4,0.04,1000,1,-1.0


#### First Model - Predict Traded

In [63]:
model_s = AdaptiveBayesianReticulum(
    prior=(1, 1),
    pruning_factor=1.01,
    n_iter=100,
    learning_rate_init=0.1,
    n_gradient_descent_steps=1,
    initial_relative_stiffness=20)

In [64]:
# extract input and target
X_train_s = simple_train[['delta_from_mid','Notional']].values
y_train_s = simple_train[['Traded']].values.ravel()
X_test_s = simple_test[['delta_from_mid','Notional']].values
y_test_s = simple_test[['Traded']].values.ravel()

In [65]:
model_s.fit(X_train_s, y_train_s, verbose=False)

origin=[4.38823339e-02, 1.70365874e-07], normal=[2.88049491e+02, 1.11830431e-03]
├─ left: origin=[7.02320661e-02, 1.62984822e-08], normal=[-4.36514617e+01, -1.01300248e-05]
│  ├─ left: origin=[-6.02174479e-02,  6.63701347e-06], normal=[ 2.01580667e+01, -2.22177068e-03]
│  │  ├─ left:  p(y)=[0.66941409 0.33058591], n=357.552839514721
│  │  └─ right:origin=[ 1.04598706e-02, -4.98942595e-09], normal=[-5.21846838e+02,  2.48924318e-04]
│  │     ├─ left: origin=[ 4.24940559e-03, -4.44301964e-10], normal=[ 1.45652364e+03, -1.52288667e-04]
│  │     │  ├─ left:  p(y)=[0.18125534 0.81874466], n=73.73310212440903
│  │     │  └─ right: p(y)=[0.04226047 0.95773953], n=42.88298404710843
│  │     └─ right:origin=[ 2.72385791e-03, -1.01534914e-08], normal=[-4.72784073e+02,  1.76235663e-03]
│  │        ├─ left:  p(y)=[0.38783997 0.61216003], n=72.49548966743993
│  │        └─ right: p(y)=[0.61072159 0.38927841], n=317.36646700790453
│  └─ right:origin=[ 9.94349507e-02, -7.49142022e-10], normal=[ 1.59744782e+02, -1.20351574e-06]
│     ├─ left: origin=[ 1.63624297e-01, -1.04321998e-06], normal=[ 5.13832899e+01, -3.27604616e-04]
│     │  ├─ left:  p(y)=[0.97552712 0.02447288], n=251.46065512998297
│     │  └─ right: p(y)=[0.91986939 0.08013061], n=592.2003539832286
│     └─ right:origin=[6.22084561e-02, 1.54886861e-11], normal=[-3.07468399e+02, -7.65536041e-08]
│        ├─ left: origin=[ 4.88958374e-03, -7.38836457e-12], normal=[ 6.24917183e+02, -9.44275876e-07]
│        │  ├─ left: origin=[ 2.73445284e-02, -3.90805085e-11], normal=[-8.44282309e+02,  1.20663927e-06]
│        │  │  ├─ left: origin=[3.01640402e-02, 4.94811429e-13], normal=[-1.99984875e+03, -3.28055530e-08]
│        │  │  │  ├─ left:  p(y)=[0.14912576 0.85087424], n=157.01913678101147
│        │  │  │  └─ right: p(y)=[0.34427341 0.65572659], n=78.60838225270348
│        │  │  └─ right:origin=[ 4.34631129e-02, -1.31534011e-10], normal=[-6.57654732e+02,  1.99028461e-06]
│        │  │     ├─ left:  p(y)=[0.40763504 0.59236496], n=156.7227282511087
│        │  │     └─ right: p(y)=[0.61188104 0.38811896], n=180.18326528558146
│        │  └─ right: p(y)=[0.05079777 0.94920223], n=177.37698368938587
│        └─ right:origin=[ 8.97962998e-02, -3.04858783e-10], normal=[-5.65148878e+02,  1.91868262e-06]
│           ├─ left: origin=[ 8.00000000e-02, -1.22136008e-23], normal=[-7.02674623e+14,  1.07277342e-07]
│           │  ├─ left: origin=[ 2.64824510e-03, -1.78363955e-11], normal=[-6.96413136e+02,  4.69046469e-06]
│           │  │  ├─ left:  p(y)=[0.54304379 0.45695621], n=35.762245388847205
│           │  │  └─ right: p(y)=[0.72553212 0.27446788], n=209.05041609630374
│           │  └─ right: p(y)=[0.80289439 0.19710561], n=178.5744503841522
│           └─ right: p(y)=[0.86853159 0.13146841], n=167.91382098250062
└─ right:origin=[2.23502917e-02, 5.25297045e-08], normal=[-8.84005738e+02, -2.07767133e-03]
   ├─ left: origin=[-4.79815783,  0.02307083], normal=[-1802.46427190,     8.66673265]
   │  ├─ left:  p(y)=[0.02189704 0.97810296], n=568.68556980475
   │  └─ right: p(y)=[0.08853291 0.91146709], n=567.1770833132216
   └─ right:origin=[7.86087280e-02, 3.82107029e-06], normal=[9.12188267e+01, 4.43403115e-03]
      ├─ left:  p(y)=[0.30617777 0.69382223], n=396.9498330739022
      └─ right: p(y)=[0.23538401 0.76461599], n=418.2841932217369

#### Second Model - Predict + and - PnL

In [66]:
model_s2 = AdaptiveBayesianReticulum(
    prior=(1, 1),
    pruning_factor=1.01,
    n_iter=100,
    learning_rate_init=0.1,
    n_gradient_descent_steps=1,
    initial_relative_stiffness=20)

In [67]:
# extract input and target
y_train_s2 = simple_train[['PnL']].values.ravel()
y_test_s2 = simple_test[['PnL']].values.ravel()

In [68]:
model_s2.fit(X_train_s, y_train_s2, verbose=False)

origin=[4.38823339e-02, 1.70365874e-07], normal=[2.88049491e+02, 1.11830431e-03]
├─ left: origin=[ 6.79589361e-02, -1.38708529e-07], normal=[ 5.03951667e+00, -1.02859754e-05]
│  ├─ left: origin=[ 1.18013645e-01, -2.34417001e-07], normal=[-1.78485135e+02,  3.54534851e-04]
│  │  ├─ left: origin=[9.00258784e-02, 2.32972387e-09], normal=[-9.99272297e+02, -2.58595480e-05]
│  │  │  ├─ left:  p(y)=[0.36399018 0.63600982], n=576.2398449400024
│  │  │  └─ right: p(y)=[0.19987146 0.80012854], n=175.14846326477007
│  │  └─ right:origin=[ 1.85568854e-01, -8.22282661e-07], normal=[-1.26019653e+02,  5.58411464e-04]
│  │     ├─ left:  p(y)=[0.12796781 0.87203219], n=180.7069220404358
│  │     └─ right: p(y)=[0.05257763 0.94742237], n=132.81663548983946
│  └─ right:origin=[ 3.16072130e-02, -3.15991854e-10], normal=[-1.64425532e+02,  1.64383772e-06]
│     ├─ left: origin=[ 1.29483678e-02, -9.13071274e-12], normal=[-4.95426948e+02,  3.49356863e-07]
│     │  ├─ left:  p(y)=[0.61231588 0.38768412], n=247.142877918728
│     │  └─ right:origin=[1.45115490e-01, 1.45283085e-09], normal=[-3.96353874e+02, -3.96811624e-06]
│     │     ├─ left:  p(y)=[0.5174237 0.4825763], n=333.63737574949187
│     │     └─ right:origin=[1.66112608e-01, 1.43043899e-09], normal=[4.74749700e+02, 4.08819348e-06]
│     │        ├─ left:  p(y)=[0.27206058 0.72793942], n=89.11077286583534
│     │        └─ right: p(y)=[0.39711518 0.60288482], n=120.23659891764434
│     └─ right:origin=[ 6.81137079e-02, -9.88325757e-10], normal=[ 1.82506214e+02, -2.64815406e-06]
│        ├─ left: origin=[ 1.61344765e-01, -2.88031056e-09], normal=[-8.42308497e+01,  1.50368068e-06]
│        │  ├─ left: origin=[ 1.21025963e-01, -1.22032752e-09], normal=[-1.22250797e+02,  1.23267775e-06]
│        │  │  ├─ left: origin=[ 9.36312326e-02, -2.59267625e-09], normal=[-6.98484146e+02,  1.93412305e-05]
│        │  │  │  ├─ left:  p(y)=[0.30968527 0.69031473], n=244.01247051477355
│        │  │  │  └─ right: p(y)=[0.20040181 0.79959819], n=120.64747941904528
│        │  │  └─ right: p(y)=[0.1686065 0.8313935], n=164.8968395155806
│        │  └─ right: p(y)=[0.06858757 0.93141243], n=150.6873184058294
│        └─ right:origin=[8.95537061e-02, 3.18872760e-09], normal=[-5.46847540e+01, -1.94715319e-06]
│           ├─ left:  p(y)=[0.38147801 0.61852199], n=358.1706562377045
│           └─ right: p(y)=[0.28135875 0.71864125], n=155.44906530670872
└─ right:origin=[4.50631396e-02, 1.12942376e-06], normal=[-1.74605847e+02, -4.37617071e-03]
   ├─ left: origin=[-7.93960600,  0.06312072], normal=[-1565.54051421,    12.44621588]
   │  ├─ left:  p(y)=[0.64023064 0.35976936], n=512.5450242268948
   │  └─ right: p(y)=[0.54635258 0.45364742], n=582.433136080446
   └─ right:origin=[ 2.78842043e-02, -5.89972800e-08], normal=[-1.12570975e+03,  2.38177186e-03]
      ├─ left:  p(y)=[0.51575053 0.48424947], n=525.09065625251
      └─ right: p(y)=[0.43373015 0.56626985], n=331.02786285375976

#### Get Probability for each test set

In [69]:
column = simple_train.columns
column

Index(['delta_from_mid', 'Notional', 'Traded', 'PnL'], dtype='object')

In [56]:
def max_pnl_price(delta_range,trade,cols,model1,model2):
    delta_range =delta_range
    sample = pd.DataFrame()
    sample['delta_from_mid'] = delta_range[::-1]
    cols = cols.drop('delta_from_mid')
    sample[cols] = trade.drop('delta_from_mid')
    
    sample['P_trade'],sample['P_pospnl']=model1.predict_proba(sample.drop(columns=['Traded','PnL']).values)[:, 1],model2.predict_proba(sample.drop(columns=['Traded','PnL']).values)[:, 1]
    
    # 0 for missing trade, 1 for trading and positive pnl, -1 for trading and negative pnl
    sample['Exp_PnL'] = (sample['P_trade']*sample['P_pospnl'])-(sample['P_trade']*(1-sample['P_pospnl']))
    pnl_argmax = np.argmax(sample['Exp_PnL'])
    return sample, sample.iloc[pnl_argmax]['delta_from_mid']

In [70]:
sample,delta = max_pnl_price(np.arange(0,1.5,0.01),simple_test.iloc[0],column,model_s,model_s2)
sample

,delta_from_mid,Notional,Traded,PnL,P_trade,P_pospnl,Exp_PnL
0,1.49,320000.0,0.0,1.0,0.080131,0.873240,0.059816
1,1.48,320000.0,0.0,1.0,0.080131,0.873301,0.059826
2,1.47,320000.0,0.0,1.0,0.080131,0.873365,0.059836
3,1.46,320000.0,0.0,1.0,0.080131,0.873432,0.059847
4,1.45,320000.0,0.0,1.0,0.080131,0.873503,0.059858
...,...,...,...,...,...,...,...
145,0.04,320000.0,0.0,1.0,0.612385,0.587360,0.106996
146,0.03,320000.0,0.0,1.0,0.679221,0.530338,0.041213
147,0.02,320000.0,0.0,1.0,0.878341,0.495101,-0.008606
148,0.01,320000.0,0.0,1.0,0.891791,0.413775,-0.153789


In [58]:
delta

0.06

In [71]:
suggested_delta_simple = []
for idx,row in simple_test.iterrows():
    sample,delta = max_pnl_price(np.arange(0,1.5,0.01),row,column,model_s,model_s2)
    suggested_delta_simple.append(delta)

#### Check PnL on test set

In [ ]:
target_mapping = {'MISSED': 0, 'DONE': 1}
side_mapping = {'Offer': -1, 'Bid': 1}

In [80]:
test.head(1)

,Time,Bond,Side,Notional,Counterparty,MidPrice,QuotedPrice,Competitors,Traded,NextMidPrice,delta_from_mid,PnL
0,30000,Bond_2,1,320000,Ctpy_3,108.2,107.89,2,0,108.32,0.31,1.0


In [88]:
test_perf = pd.DataFrame()
test_perf[['Traded','Side','QuotedPrice','NextMidPrice','PnL']]=test[['Traded','Side','QuotedPrice','NextMidPrice','PnL']]
test_perf['delta']=suggested_delta_simple
test_perf['My_Price']=test['MidPrice']-(test_perf['delta']*test_perf['Side'])

In [89]:
test_perf.head(3)

,Traded,Side,QuotedPrice,NextMidPrice,PnL,delta,My_Price
0,0,1,107.89,108.32,1.0,0.06,108.14
1,1,1,84.76,84.74,-1.0,0.04,84.75
2,1,1,84.73,84.61,-1.0,0.04,84.70


In [93]:
performance = 0
unknown_idx = []
for idx,row in test_perf.iterrows():
    if (row['Traded']==1) & (row['Side']==1) & (row['My_Price']<row['QuotedPrice']):
        if row['My_Price'] < row['NextMidPrice']:
            performance+=1
        else:
            performance -=1
    elif (row['Traded']==1) & (row['Side']==-1) & (row['My_Price']>row['QuotedPrice']):
        if row['My_Price'] > row['NextMidPrice']:
            performance+=1
        else:
            performance -=1
    elif (row['Traded']==0) & (row['Side']==1) & (row['My_Price']<row['QuotedPrice']):
        unknown_idx.append(idx)
    elif (row['Traded']==0) & (row['Side']==-1) & (row['My_Price']>row['QuotedPrice']):
        unknown_idx.append(idx)
performance

73

In [94]:
test_perf['PnL'].sum()

290.0

## All Features

In [5]:
def add_features(df):
    df['delta_from_mid'] = abs(df['MidPrice'] - df['QuotedPrice'])
    df['Traded'] = df['Traded'].map(target_mapping)
    df['Side'] = df['Side'].map(side_mapping)
    cols = ['Counterparty']    
    one_hot_encode = pd.get_dummies(df[cols])
    df = df.join(one_hot_encode)
    df['Notional_cuts'] = pd.cut(df['Notional'], [0, 100000, 1000000, 10000000, 50000000], labels=[0, 1, 2, 3], include_lowest=True, right=True)
    df['PnL']=((df['NextMidPrice']-df['QuotedPrice']>0).astype(float)*2-1)*df['Side']
    cols_to_drop = ['Time','Bond','Side','Counterparty','NextMidPrice','Notional','MidPrice','QuotedPrice']
    return df.drop(columns=cols_to_drop)

In [6]:
train_all = add_features(training)
test_all = add_features(test)
train_all.head()

,Competitors,Traded,delta_from_mid,Counterparty_Ctpy_0,Counterparty_Ctpy_1,Counterparty_Ctpy_2,Counterparty_Ctpy_3,Notional_cuts,PnL
0,1,0,0.24,1,0,0,0,2,1.0
1,1,1,0.01,0,1,0,0,0,1.0
2,1,0,0.10,0,1,0,0,0,-1.0
3,4,1,0.02,1,0,0,0,0,-1.0
4,2,1,0.04,0,0,0,1,0,-1.0


#### First Model - Predict Traded

In [10]:
# extract input and target
X_train = train_all.drop(['PnL','Traded'],axis=1).values
y_train = train_all[['Traded']].values.ravel()
X_test = test_all.drop(['PnL','Traded'],axis=1).values
y_test = test_all[['Traded']].values.ravel()

In [11]:
# train model
model = AdaptiveBayesianReticulum(
    prior=(1, 1),
    pruning_factor=1.01,
    n_iter=100,
    learning_rate_init=0.1,
    n_gradient_descent_steps=1,
    initial_relative_stiffness=20)

t0 = dt.datetime.utcnow()
model.fit(X_train, y_train, verbose=False)
t1 = dt.datetime.utcnow()

#### Second Model - Predict + and - PnL

In [14]:
# extract input and target
X_train_p = train_all.drop(['PnL','Traded'],axis=1).values
y_train_p = train_all[['PnL']].values.ravel()
X_test_p = test_all.drop(['PnL','Traded'],axis=1).values
y_test_p = test_all[['PnL']].values.ravel()

In [15]:
# train model
model_p = AdaptiveBayesianReticulum(
    prior=(1, 1),
    pruning_factor=1.01,
    n_iter=100,
    learning_rate_init=0.1,
    n_gradient_descent_steps=1,
    initial_relative_stiffness=20)

t0 = dt.datetime.utcnow()
model_p.fit(X_train_p, y_train_p, verbose=False)
t1 = dt.datetime.utcnow()

#### Get Probability for each test set

In [21]:
columns = test_all.columns
columns

Index(['Competitors', 'Traded', 'delta_from_mid', 'Counterparty_Ctpy_0',
       'Counterparty_Ctpy_1', 'Counterparty_Ctpy_2', 'Counterparty_Ctpy_3',
       'Notional_cuts', 'PnL'],
      dtype='object')

In [36]:
sample,delta = max_pnl_price(np.arange(0,1.5,0.01),test_all.iloc[0],columns,model,model_p)
sample

,delta_from_mid,Competitors,Traded,Counterparty_Ctpy_0,Counterparty_Ctpy_1,Counterparty_Ctpy_2,Counterparty_Ctpy_3,Notional_cuts,PnL,P_trade,P_pospnl,Exp_PnL
0,1.49,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.02997,0.929963,0.025772
1,1.48,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.02997,0.929963,0.025772
2,1.47,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.02997,0.929963,0.025772
3,1.46,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.02997,0.929963,0.025772
4,1.45,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.02997,0.929963,0.025772
...,...,...,...,...,...,...,...,...,...,...,...,...
145,0.04,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.02997,0.929963,0.025772
146,0.03,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.02997,0.929963,0.025772
147,0.02,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.02997,0.929963,0.025772
148,0.01,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.02997,0.929963,0.025772


In [39]:
suggested_delta = []
for idx,row in test_all.iterrows():
    sample,delta = max_pnl_price(np.arange(0,1.5,0.01),row,columns,model,model_p)
    suggested_delta.append(delta)